# Additional Modelling Challenges

## Balanced Academic Curriculum Problem

Design an academic curriculum so that the academic load of each period is balanced.

The curriculum must obey the following administrative and academic regulations:

- Academic curriculum: an academic curriculum is defined by a set of courses and a set of prerequisite relationships among them.
- Number of periods: courses must be assigned within a maximum number of academic periods.
- Academic load: each course has associated a number of credits or units that represent the academic effort required to successfully follow it.
- Minimum academic load: a minimum number of academic credits per period is required to consider a student as full time.
- Minimum number of courses: a minimum number of courses per period is required to consider a student as full time.
- Maximum academic load: a maximum number of academic credits per period is allowed in order to avoid overload.
- Maximum number of courses: a maximum number of courses per period is allowed in order to avoid overload.

The goal is to assign a period to every course in a way that the minimum and maximum academic load for each period, the minimum and maximum number of courses for each period, and the prerequisite relationships are satisfied.

An optimal balanced curriculum minimises the maximum academic load for all periods.

In [1]:
import cpmpy as cp
import enum

# Parameters
# Academic curriculum: an academic curriculum is defined by a set of courses and a set of prerequisite relationships among them.
class Course(enum.Enum):
    CS = "Cognitive Science"
    ETH = "Ethics"
    FAI = "Fundamentals of Symbolic AI"
    KR = "Knowledge Representation"
    IML = "Introduction to ML"
    CV = "Computer Vision"
    DL = "Deep Learning"
    NLP = "Natural Language Processing"
    RL = "Reinforcement Learning"
    DRL = "Deep Reinforcement Learning"
    SVM = "Support Vector Machines"

course_list = list(Course)
prerequisites = {Course.CS: [], Course.ETH: [], Course.FAI: [], Course.KR: [Course.FAI], Course.IML: [],
                 Course.CV: [Course.IML], Course.DL: [Course.IML], Course.NLP: [Course.IML], Course.RL: [Course.IML],
                 Course.DRL: [Course.DL, Course.RL], Course.SVM: [Course.IML]}

# Number of periods: courses must be assigned within a maximum number of academic periods.
max_n_periods = 4

# Academic load: each course has associated a number of credits or units that represent the academic effort required to successfully follow it.
credits = {Course.CS: 5, Course.ETH: 3, Course.FAI: 6, Course.KR: 5, Course.IML: 6, Course.CV: 4, Course.DL: 6,
           Course.NLP: 4, Course.RL: 4, Course.DRL: 4, Course.SVM: 4}
print("Total credits:", sum(value for value in credits.values()))

# Minimum academic load: a minimum number of academic credits per period is required to consider a student as full time.
# Minimum number of courses: a minimum number of courses per period is required to consider a student as full time.
min_credits_per_period = 8
min_courses_per_period = 2

# Maximum academic load: a maximum number of academic credits per period is allowed in order to avoid overload.
# Maximum number of courses: a maximum number of courses per period is allowed in order to avoid overload.
max_credits_per_period = 18
max_courses_per_period = 4

model = cp.Model()

# Decision variables
# The goal is to assign a period to every course in a way that the minimum and maximum academic load for each period,
# the minimum and maximum number of courses for each period, and the prerequisite relationships are satisfied.
curriculum = cp.boolvar(shape=(len(Course), max_n_periods))

# Constraints
# Academic curriculum: an academic curriculum is defined by a set of courses and a set of prerequisite relationships among them.
for course_idx, course in enumerate(Course):
    model += cp.sum(curriculum[course_idx, :]) == 1

    for prereq in prerequisites[course]:
        prereq_idx = course_list.index(prereq)

        for period in range(max_n_periods):
            model += [curriculum[prereq_idx, period].implies(~curriculum[course_idx, earlier_period]) for earlier_period in range(period)]
            model += curriculum[prereq_idx, period].implies(~curriculum[course_idx, period])

# Minimum and maximum number of courses and academic load.
for period in range(max_n_periods):
    model += min_courses_per_period <= cp.sum(curriculum[:, period])
    model += cp.sum(curriculum[:, period]) <= max_courses_per_period

    period_credits = cp.sum([curriculum[course_idx, period] * credits[course] for course_idx, course in enumerate(Course)])
    model += min_credits_per_period <= period_credits
    model += period_credits <= max_credits_per_period

# An optimal balanced curriculum minimises the maximum academic load for all periods.
model.minimize(cp.Maximum([cp.sum(curriculum[:, p]) for p in range(max_n_periods)]))

if model.solve():
    print("Solution found!")
    print(curriculum.value())

    print("\nCurriculum Schedule:")
    print("-" * 80)
    header = "Course (Credits)".ljust(25)
    for period in range(max_n_periods):
        header += f"| Period {period+1} "
    print(header)
    print("-" * 80)

    for course_idx, course in enumerate(Course):
        row = f"{course.name} ({credits[course]})".ljust(25)
        for period in range(max_n_periods):
            row += f"| {'X' if curriculum.value()[course_idx][period] else ' ':^9}"
        print(row)

    print("-" * 80)

    courses_row = "Courses per period".ljust(25)
    for period in range(max_n_periods):
        period_courses = sum(curriculum.value()[c][period] for c in range(len(Course)))
        courses_row += f"| {period_courses:^8} "
    print(courses_row)
    credits_row = "Credits per period".ljust(25)
    for period in range(max_n_periods):
        period_credits = sum(credits[course] * curriculum.value()[c][period] for c, course in enumerate(Course))
        credits_row += f"| {period_credits:^8} "
    print(credits_row)
else:
    print("No solution found.")

Total credits: 51
Solution found!
[[False False False  True]
 [ True False False False]
 [ True False False False]
 [False False  True False]
 [ True False False False]
 [False False  True False]
 [False False  True False]
 [False  True False False]
 [False  True False False]
 [False False False  True]
 [False  True False False]]

Curriculum Schedule:
--------------------------------------------------------------------------------
Course (Credits)         | Period 1 | Period 2 | Period 3 | Period 4 
--------------------------------------------------------------------------------
CS (5)                   |          |          |          |     X    
ETH (3)                  |     X    |          |          |          
FAI (6)                  |     X    |          |          |          
KR (5)                   |          |          |     X    |          
IML (6)                  |     X    |          |          |          
CV (4)                   |          |          |     X    |     